# Домашнее задание №8: Изучение и учёт движения геодезических пунктов

При многократном определении прямоугольных геоцентрических координат геодезического пункта наблюдаются неслучайные их изменения со временем. Другими словами геодезические пункты на поверхности Земли находятся в постоянном, и зачастую в непростом, движении. Список значений одной и той же величины, определённой в разные моменты времени, называется временной ряд. Пример временных рядов: 1) Временной ряд прямоугольных геоцентрических координат геодезического пункта; 2) Временной ряд значений силы тяжести на гравиметрическом пункте; 3) Временной ряд температуры вблизи геодезического пункта и т.д. Пример графика временного ряда прямоугольных геоцентрических координат приведён на рисунке ниже. 

![ ](https://zenodo.org/api/iiif/record:10902962:SLH1_4y_point_XYZ.png/full/!1400,1400/0/default.png "Временной ряд геоцентрических координат пункта Салехард (без тренда)")

## Линейная регрессия

В первом приближении движение геодезического пункта можно представить в виде линейной функции вида:

$$\tag{1}
\vec{r}_t = \vec{V_r}(t - t_0) + \vec{r}_{t_0},
$$

где $\vec{r}_t - $ вектор геоцентрических координат геодезического пункта на момент $t$; $\vec{V_r} - $ вектор скоростей изменения геоцентрических координат геодезического пункта; $\vec{r}_{t_0} - $ вектор геоцентрических координат геодезического пункта на момент $t_0$.

В развёрнутом виде уравнение (1) можно записать в виде системы уравнений для каждой координаты:

$$
\begin{equation}\tag{2}
 \begin{cases}
     X_t = V_x(t - t_0) + X_{t_0} \\
     Y_t = V_y(t - t_0) + Y_{t_0} \\
     Z_t = V_z(t - t_0) + Z_{t_0} \\
 \end{cases}
\end{equation}
$$

Очевидно, что для нахождения скорости достаточно двух измерений. Однако при наличии избыточных измерений следует применить метод наименьших квадратов, таким образом, получить оценку линейной регрессии.

Для реализации метода наименьших квадратов параметрическим способом на первом шаге составляют параметрические уравнения, которые устанавливают связь между измеренными и определяемыми величинами. В нашем случае это уравнения (2). 

Далее находятся три исходные матрицы: $A - $ матрица коэффициентов параметрических уравнений; $P - $ весовая матрица; $L - $ вектор свободных членов, содержащий данные измерений.

### Матрица коэффициентов $A$
Матрица коэффициентов $A$ находится путём дифференцирования уравнений связи по определяемым величинам.
Рассмотрим уравнение для координаты $X$:

$$
X_t = V_x(t - t_0) + X_{t_0},
$$

Здесь неизвестными величинами являются скорость изменения данной координаты - $V_x$ и значение координаты $X$ на опорную эпоху $t_0$ - $X_{t_0}$. 

Таким образом матрица коэффициентов $A$ будет иметь следующий вид:

$$
A_{n\times k} = 
\begin{pmatrix}
\dfrac{\partial X_{t_1}}{\partial V_x} & \dfrac{\partial X_{t_1}}{\partial X_{t_0}} \\
\dfrac{\partial X_{t_2}}{\partial V_x} & \dfrac{\partial X_{t_2}}{\partial X_{t_0}} \\
\dots & \dots \\
\dfrac{\partial X_{t_n}}{\partial V_x} & \dfrac{\partial X_{t_n}}{\partial X_{t_0}} \\
\end{pmatrix}
=
\begin{pmatrix}
(t_1 - t_0) & 1 \\
(t_2 - t_0) & 1 \\
\dots & \dots \\
(t_i - t_0) & 1 \\ \\
\end{pmatrix},
$$

где $n - $ количество строк во временном ряду; $k - $ количество определяемых величин.

### Весовая матрица $P$

Весовая матрица $P$ отражает информацию о модели шумов (погрешностей) исходных данных. 

В случае если координаты во временном ряду равноточные, то весовая матрица будет равна единичной:

$$
P_{n\times n} =
\begin{pmatrix}
	1 & 0 & 0 & \cdots & 0 \\
	0 & 1 & 0 & \cdots & 0 \\
	\cdots & \cdots & \cdots & \cdots & \cdots\\
	0 & 0 & 0 & \cdots & 1 \\
\end{pmatrix}.
$$

В случае если измерения неравноточные и некоррелированные (ещё можно сказать что эта ситуация соответствует присутствию в данных белого шума) на главной диагонали весовой матрицы будут располагаться веса измерений. Назначение веса может быть произвольным, однако классическим является способ назначения веса равным обратным погрешности измерений - $\sigma_{X_n}^2$. 

Например, для координаты $X$ в случае белого шума весовая матрица будет выглядеть следующим образом:

$$
P = 
	\begin{pmatrix}
	\frac{1}{\sigma_{X_1}^2} & 0 & 0 & \cdots & 0 \\
	0 & \frac{1}{\sigma_{X_2}^2} & 0 & \cdots & 0 \\
	\cdots & \cdots & \cdots & \cdots & \cdots\\
	0 & 0 & 0 & \cdots & \frac{1}{\sigma_{X_n}^2} \\
	\end{pmatrix}
$$


В случае если измерения неравноточные и коррелированные (можно сказать что эта ситуация соответствует присутствию в данных как белого шума, так и цветных шумов) весовая матрица будет заполнена значениями, как на главной диагонали, так и вне неё:

$$
P = 
	\begin{pmatrix}
	\frac{1}{\sigma_{X_1}^2} & K_{X_1X_2} & K_{X_1X_3} & \cdots & K_{X_1X_n} \\
	K_{X_2X_1} & \frac{1}{\sigma_{X_2}^2} & K_{X_2X_3} & \cdots & K_{X_2X_n} \\
	\cdots & \cdots & \cdots & \cdots & \cdots\\
	K_{X_nX_1} & K_{X_nX_2} & K_{X_nX_3} & \cdots & \frac{1}{\sigma_{X_n}^2} \\
	\end{pmatrix}.
$$

Однако нахождение зависимостей (ковариаций) между измеренными величинами - $K_{X_iX_j}$ является нетривиальной задачей и требует привлечения дополнительных данных и гипотез.



### Вектор свободных членов L

Вектор свободных членов $L$ сожержит разности между измеренными величинами и их априорными (приближенными/предполагаемыми) значениями. Для нахождения априорных значений измеренных величин следует задать приближенные значения определчемых параметров. 

Например для координаты $X$ в случае линейной функции её изменения: $X_t = V_x(t - t_0) + X_{t_0}$ определяемыми параметрами являются скорость изменения координаты $V_x$ и координата на опорную эпоху $X_{t_0}$. 

Обозначим приближенные значения определяемых параметров следующим образом: $V_x^{(0)}$ и $X_{t_0}^{(0)}$.

Тогда априорное значение измеренной координаты будет вычисляться следующим образом:

$$
X_t^{(0)} = V_x^{(0)}(t - t_0) + X_{t_0}^{(0)}
$$

Для линейной апроксимации геоцентрических координат априорные значения вычисляемых параметров можно принять равными нулю. Тогда в таком случае для координаты $X$ вектор свободных членов будет выглядеть следующим образом:

$$
L_{1\times n} = 
	\begin{pmatrix}
	(X_t^{(0)} - X_{t_1}) \\
	(X_t^{(0)} - X_{t_2}) \\
	\cdots \\
	(X_t^{(0)} - X_{t_n}) \\
	\end{pmatrix}
= 
\begin{pmatrix}
	(0 - X_{t_1}) \\
	(0 - X_{t_2}) \\
	\cdots \\
	(0 - X_{t_n}) \\
	\end{pmatrix}
 =
\begin{pmatrix}
	-X_{t_1} \\
	-X_{t_2} \\
	\cdots \\
	-X_{t_n} \\
\end{pmatrix} 
$$


### Решение метода наименьших квадратов

После того как были составлены три исходные матрицы появляется возможность вычислить поправки к априорным значениям определяемых параметров:

$$
\Delta X = -(A^TPA)^{-1}A^TPL
$$

Если в качестве априорных значений был принят нуль, то вектор $\Delta X$ будет содержать значения определяемых параметров.

Например для координаты $X$:

$$
\Delta X =
\begin{pmatrix}
V_X\\
X_{t_0}
\end{pmatrix}
$$

Ковариационная матрица определяемых величин вычисляется по формуле:

$$
Q = (A^TPA)^{-1}
$$

На главной диагонали ковариационной матрицы $Q$ находятся дисперссии (средние квадратические погрешности в квадрате) определяемых величин. Так, в случае линейной аппроксимации изменения, и при назначении весов как $\frac{1}{\sigma_{X_n}}$, на примере координаты $X$:

$$
m_{V_X} = \sqrt{Q_{11}}
$$

$$
m_{X_{t_0}} = \sqrt{Q_{22}}
$$

## Система координат ENU

Движение геодезических пунктов принято представлять в топоцентрической системе координат ENU (E-East, N-North, U-Up). При таком подходе легко разделить горизонтальное движение - EN и вертикальное - U.

![ ](https://zenodo.org/api/iiif/record:10907953:coordinate_system_ENU.png/full/!500,500/0/default.png "Система координат ENU")


Трансформировать координаты из XYZ в ENU можно по следующей формуле:

$$
\begin{pmatrix}
E\\
N\\
U
\end{pmatrix}
=
\begin{pmatrix}
-\sin L & \cos L & 0 \\
-\sin B\cos L & -\sin B\sin L & \cos B \\
\cos B\cos L & \cos B\sin L & \sin B
\end{pmatrix}
\begin{pmatrix}
X - X_0 \\
Y - Y_0 \\
Z - Z_0 \\
\end{pmatrix},
$$

где $X_0, Y_0, Z_0 - $ начало системы ENU. В случае временных рядов координат в качестве начала системы ENU может быть выбрана первая эпоха временного ряда.


## Задание 1
Даны два пункта с установленным на них постоянно действующим спутниковым геодезическим оборудованием (GNSS). Проводятся многолетние непрерывные измерения, в результате обработки которых получен временной ряд геоцентрических координат пункта $X, Y, Z$ c дискретностью 1 сутки.

**Задание 1**. Построить графики временных рядов координат пунктов в геоцентрической X, Y, Z и горизонтной E, N, U системах (отдельно по каждой координате).

**Задание 2**. Из обработки всего временного ряда определить скорость движения пунктов по каждой координате в системах $X, Y, Z$ и $E, N, U$ в мм/год с оценкой точности. Нанести линию
тренда на графики из предыдущего пункта. Применяя правило сложения векторов, найти величину и направление (азимут) горизонтальной скорости.


**Выбор исходных данных.** Временные ряды координат для двух самостоятельно выбранных пунктов необходимо получить с сайта Невадской геодезической лаборатории:

Список: http://geodesy.unr.edu/NGLStationPages/GlobalStationList

Карта: http://geodesy.unr.edu/NGLStationPages/gpsnetmap/GPSNetMap.html

Для чтения файла с временным рядом геоцентрических координат с данного сайта воспользуемся библиотекой pandas:

In [ ]:
import pandas as pd

data_df = pd.read_csv('http://geodesy.unr.edu/gps_timeseries/txyz/IGS14/0ABY.txyz2',
                      sep='\s+', header=None, usecols=[0, 2, 3, 4, 5, 6, 7, 8],
                      names=['ID', 'Time', 'X', 'Y', 'Z', 'sigma_X', 'sigma_Y', 'sigma_Z'])

print(data_df)

В коде выше в ссылке следует заменить "0ABY" на ID выбранного вами пункта. В результате чего в переменную "data_df" помещается объект бибилиотеки pandas называемый DataFrame, который представляет из себя таблицу с данными. Далее с данными из этой таблицы легко проводить различные манипуляции. С этим можно познакомиться на вводной документации к pandas: https://pandas.pydata.org/docs/user_guide/10min.html 

Критерии для выбора пунктов следующие:

1. Первая буква в четырёхсимвольном коде первого пункта должна совпадать с первой буквой фамилии студента, первая буква в четырёхсимвольном коде второго пункта должна совпадать с первой буквой имени студента.

2. На обоих пунктах должно быть не менее 5 лет измерений. Причем, достаточно полных, хотя могут быть и пропуски. Интервалы по обоим пунктам не обязательно должны перекрываться, но желательно, что бы у них был хотя бы 1 общий год.

3. Расстояние между пунктами должно быть не менее 1000 km. Еще лучше, что бы они располагались на разных литосферных плитах.

## Модель движения тектонических плит

Главной причиной линейного изменения координат геодезических пунктов является тектоническое движение литосферных плит. Это движение часто моделируют на основе теоремы вращения Эйлера. Данная теорема гласит: *Движение любого тела по поверхности сферы можно представить в виде вращения вокруг оси, проходящей через её центр и пересекающей поверхность сферы в двух точках, или полюсах (Эйлера).*

Данная теорема поясняется на рисунке ниже

![ ](https://zenodo.org/api/iiif/record:10908401:Eulerpol-ru.png/full/!500,500/0/default.png "Теорема вращения Эйлера")

Важным параметром моделей движения тектонических плит является подробность деления земной поверхности на блоки. Установление границ тектонических плит требует использования комплексного набора исходных данных (геодезия, геофизика, геология и др.) 

Таким образом модели движения тектонических плит задают:

1. Число и границы тектонических плит
2. Координаты полюса Эйлера для каждой плиты
3. Угловую скорость вращения каждой плиты

Зная эти параметры можно вычислить составляющие горизонтальной скорости пункта на поверхности Земли:

$$\tag{3}
V_E = R\omega\cos\varphi(\sin\varphi_P - \cos\varphi_P\tan\varphi\cos(\lambda_P - \lambda)),
$$

$$\tag{4}
V_N = R\omega\cos\varphi_P\sin(\lambda - \lambda_P),
$$

где $R = 6371\,000$\,м - средний радиус Земли; $\omega - $ угловая скорость вращения плиты; $\varphi, \lambda - $ координаты пункта; $\varphi_P, \lambda_P - $ координаты полюса Эйлера; 

Значение горизонтальной скорости и её направление (азимут) вычисляется по формулам:

$$
v = \sqrt{v_E^2 + v_N^2}
$$

$$
A = \arctan\frac{V_E}{V_N}
$$

При вычислении арктангенса нужно правильно учесть знаки в числителе и знаменателе аргумента (то есть воспользоваться функцией ATAN2, а не просто ATAN).

## Задание 2

1. Пользуясь моделью движения тектонических плит NNR-MORVEL56, получить оценки скоростей для двух пунктов из предыдущего задания. Вычислить величину горизонтальной скорости и её азимут.
2. Проверить свои вычисления с помощью сервиса UNAVCO Plate Motion Calculator

Координаты полюсов Эйлера и угловые скорости каждой плиты для модели NNR–MORVEL56 можно взять отсюда: http://www.geology.wisc.edu/~chuck/MORVEL/NNRMRVL56_AVs.html

**Важно!** не забыть что при вычислениях по формулам (3) и (4) координаты в функции $\sin$ и $\cos$ должны передаваться в радианах, а угловая скорость вращения $\omega$ должна использоваться в размерности радиан в год. На сайте модели NNR–MORVEL56 (http://www.geology.wisc.edu/~chuck/MORVEL/NNRMRVL56_AVs.html), координаты полюса Эйлера заданы в градусах, а угловая скорость вращения задана в размерности градусы в миллион лет.


### UNAVCO Plate Motion Calculator
Онлайн калькулятор скоростей движения тектонических плит находится по адресу: https://www.unavco.org/software/geodetic-utilities/plate-motion-calculator/plate-motion-calculator.html

Здесь всё довольно интуитивно понятно.

1. Необходимо задать геодезические координаты ИЛИ прямоугольные геоцентрические координаты;
2. Выбрать модель NUVEL 1A;
3. Назначить плиту. Здесь можно оставить Auto, тогда плита будет определена автоматически по модели NUVEL 1A. Надо иметь ввиду, что в других моделях деление плит может быть иным, но здесь при автоматическом назначении плиты будет использовано деление именно из NUVEL 1A. Эту информацию можно использовать для вычисления по модели MORVEL;
4. Reference должен быть NNR — No net rotation, то есть ни одна из плит не будет опорной, а вычисления будут в общеземной системе;
5. И дальше в конце выбрать формат и координаты для вывода информации.